This code works through the GEDI subsetter version 0.6.0. This converts a geojson of tiles over a study area to temp geojson files to feed into the subsetter

In [22]:
#Open the MAAP host
from maap.maap import MAAP
maap = MAAP(maap_host='api.ops.maap-project.org')


In [113]:
#Import numpy for making column inputs easier
import numpy as np
import geopandas as gpd
import shapely

In [137]:
#Load in Tiled AOI
tiles = gpd.read_file("https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/EEZBounds/WestAfricaGrid.geojson")

print(tiles.info)

<bound method DataFrame.info of          id    random                                           geometry
0      8,-7  0.685510  POLYGON ((8.00000 -7.00000, 9.00000 -7.00000, ...
1      9,-7  0.452251  POLYGON ((9.00000 -7.00000, 10.00000 -7.00000,...
2     12,-7  0.231314  POLYGON ((12.00000 -7.00000, 13.00000 -7.00000...
3      8,-6  0.597992  POLYGON ((8.00000 -6.00000, 9.00000 -6.00000, ...
4      9,-6  0.995462  POLYGON ((9.00000 -6.00000, 10.00000 -6.00000,...
..      ...       ...                                                ...
146  -17,20  0.612239  POLYGON ((-17.00000 20.00000, -16.00000 20.000...
147  -16,20  0.167271  POLYGON ((-16.00000 20.00000, -15.00000 20.000...
148  -18,21  0.473960  POLYGON ((-18.00000 21.00000, -17.00000 21.000...
149  -17,21  0.156771  POLYGON ((-17.00000 21.00000, -16.00000 21.000...
150  -16,21  0.512777  POLYGON ((-16.00000 21.00000, -15.00000 21.000...

[151 rows x 3 columns]>


In [99]:
#Set index to the correct column name, in this case "random"
tiles.set_index('random',inplace=True)

In [100]:
#Test a single tile
tiles.loc[0.6855097362903453]

id                                              8,-7
geometry    POLYGON ((8 -7, 9 -7, 9 -6, 8 -6, 8 -7))
Name: 0.6855097362903453, dtype: object

Use "random" column for ID to generate features to tile over

In [101]:
#List out the individual aois
list_aois = tiles.index.unique().tolist() #tiles[tiles.columns[1]].values.tolist()
print(list_aois)

[0.6855097362903453, 0.4522512836485828, 0.23131388830190724, 0.5979918962060293, 0.9954619410018374, 0.7363462239298356, 0.16955097115328577, 0.17510753017660985, 0.3925692959970333, 0.5752152674405157, 0.7412455810081872, 0.7392697493154132, 0.4653903570931611, 0.8354018764581234, 0.38600958865659807, 0.23490126284236712, 0.6936479469170777, 0.895062222650941, 0.789772170178162, 0.5160985973254504, 0.9244779703796152, 0.5150738620210101, 0.894109796377685, 0.06989256159489654, 0.6310652493506097, 0.8557841478391265, 0.3657585583436155, 0.7485440110765288, 0.5672959857440157, 0.33153070200335955, 0.39138004097717916, 0.5822984975127008, 0.4411987903729985, 0.5549448029680308, 0.1657510400319916, 0.658522680224497, 0.5666193031784508, 0.4415834231141531, 0.9917320562143129, 0.5326880936290095, 0.9621639521726443, 0.07476782962538708, 0.7317576801263533, 0.8968646730286827, 0.14280037807636703, 0.582712155141294, 0.03319394615667215, 0.6036670510954576, 0.023375478336133515, 0.175617543

In [114]:
aoiTest=shapely.to_geojson(tiles.loc[0.6855097362903453,"geometry"])
print(aoiTest)

{"type":"Polygon","coordinates":[[[8.0,-7.0],[9.0,-7.0],[9.0,-6.0],[8.0,-6.0],[8.0,-7.0]]]}


In [74]:
#Create empty variables to fill with all 29 appearances of cover_z* and pai_z* 
variables = []
for n in np.arange(0, 30,1):
    variables.append('cover_z' + str(n))
    variables.append('pai_z' + str(n))


For several AOI's run this code:

In [75]:
#Set up job results to print output during run

import xml.etree.ElementTree as ET
from urllib.parse import urlparse

def job_status_for(job_id: str) -> str:
    response = maap.getJobStatus(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)
    status_element = root.find('.//{http://www.opengis.net/wps/2.0}Status')
    
    return status_element.text

def job_result_for(job_id: str) -> str:
    response = maap.getJobResult(job_id)
    response.raise_for_status()
    
    root = ET.fromstring(response.text)

    return root.find('.//{http://www.opengis.net/wps/2.0}Data').text

def to_job_output_dir(job_result_url: str) -> str:
    url_path = urlparse(job_result_url).path
    # The S3 Key is the URL path excluding the `/{username}` prefix
    s3_key = "/".join(url_path.split("/")[2:])

    return f"/projects/my-private-bucket/{s3_key}"

In [115]:
#Import geojson and tempfile for creating temp files for the subsetter

from geojson import Point, Feature, FeatureCollection, dump

import tempfile

In [128]:
#def write_json(shape):
   # feature is a shapely geometry type
#        geom_in_geojson = geojson.Feature(geometry=shape, properties={})
#        tmp_file = tempfile.mkstemp(suffix='.geojson')
#        with open(tmp_file[1], 'w') as outfile:
#            geojson.dump(geom_in_geojson, outfile)
#        return tmp_file[1]

In [140]:
#list_aois = tiles['random'].unique().tolist()
#list_aois

In [142]:
#for aoi in list_aois[:1]:
    #shape = shapely.to_geojson(tiles.loc[aoi,"geometry"])
    #f = write_json(shape)
#    tile = tiles[tiles['random']==aoi]
    #print(type(tile))
#    tile.to_file('temp.geojson')
    

<class 'geopandas.geodataframe.GeoDataFrame'>


In [ ]:
#Set up input dictionary and import backoff to display run results
#First part of the for loop uses the "random" column to search through the tiles geojson to 
#create temp files one at a time to set the AOI
#################################

for aoi in list_aois: 
    
    
    tile = tiles[tiles['random']==aoi]
    tile.to_file('temp.geojson')
    
    inputs = dict(
       aoi="https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/MAAP_GEDIAnalysis/temp.geojson",
       doi="L2B",
       lat="geolocation/lat_lowestmode",
       lon="geolocation/lon_lowestmode",
       beams="all",
       columns="rh100, cover,land_cover_data/landsat_treecover, pai,fhd_normal,"+",".join(variables),
       query="l2a_quality_flag == 1 and l2b_quality_flag == 1 and sensitivity > 0.95",
       limit = 1000,
       temporal="-",
       output="-"
    )
    result = maap.submitJob(
        identifier="gedi-subset",
        algo_id="gedi-subset_ubuntu",
        version="0.6.0",
        queue="maap-dps-worker-32gb",
        username="abarenblitt",
    **inputs
    )
    
    job_id = result["job_id"]
    print(job_id)
    
try:
    import backoff
except:
    job_status = job_status_for(job_id)
else:
    # Check job status every 2 minutes
    @backoff.on_predicate(
        backoff.constant,
        lambda status: status not in ["Deleted", "Succeeded", "Failed"],
        interval=120,
    )
    def wait_for_job(job_id: str) -> str:
        return job_status_for(job_id)

    job_status = wait_for_job(job_id)
    job_status

2b9644bc-f429-4bab-9a9c-bc37d99d0825
97ecde26-b976-4ae0-a619-febe38252898
6095cc48-7f11-4c1c-92ca-9cc196d0e4cf
57301614-9088-4dc9-b718-6718a9b64d92
5d9f33ca-f0a8-49e7-9ab9-99a485e745c0
00257b35-32d1-45b2-b01c-c9af9dbe8313
dc53fc02-6ded-4466-ac79-29a9aa7e76ca
e89a3d5b-5825-4885-9c53-f55dfe111877
bb864ad5-c571-4abe-9d17-55c8953428a1
aa50847e-ab59-463b-8e91-9492bf60c3c3
053ef475-56f7-4d28-82bf-a45e8c679940
66894945-fddb-41d6-a790-9ca344b7b8ab
b42dabdf-006b-49fb-aeeb-a91fd0913ccd
6e20c105-04e4-4239-a40a-1afdc063a298
87faf2f6-daad-4d85-8bad-0b61e56f781d
21fc0756-1598-4cad-baa9-5fde5a86cce7
497614ea-4cdc-4d64-a19d-253373829002
38307f42-32f7-4bb4-bfbd-6da839f6b16e
45784887-a8ee-4b2b-a6c0-cffbe2256154
78b3e3bb-d0b4-4b09-85c2-8758c05acb59
9035ac64-7314-4e11-aa05-7159680243b8
e4f0cce5-1214-4b65-8c22-738e3358b57b
6ac7abbc-e081-4f58-989b-bb8506cf081c
cf0d641e-fe57-4a48-8308-ac081c9be02a
58ac520d-61f0-4f3b-9198-e5a7ec73fd43
b33e52fa-5636-48ca-a810-58622548568c
1bb9ec9b-ca4c-435a-a6d2-687ea4224b9d
1

INFO:backoff:Backing off wait_for_job(...) for 48.9s (Accepted)


9a90ea3d-60a1-4cb2-9b5e-a3886a603fc0


INFO:backoff:Backing off wait_for_job(...) for 21.8s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 108.2s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 24.6s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 9.5s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 73.0s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 41.8s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 90.0s (Accepted)
INFO:backoff:Backing off wait_for_job(...) for 32.6s (Accepted)


In [95]:
job_id = "a4596ff3-8bfc-4d03-a9dc-3cb4883b9d59"

job_status

'Failed'

In [96]:
#Get job result full text to bug Alex with
 
maap.getJobResult(job_id).text

'<wps:Result xmlns:ows="http://www.opengis.net/ows/2.0" xmlns:schemaLocation="http://schemas.opengis.net/wps/2.0/wps.xsd" xmlns:wps="http://www.opengis.net/wps/2.0" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><wps:JobID>a4596ff3-8bfc-4d03-a9dc-3cb4883b9d59</wps:JobID><wps:Output id="traceback"><wps:Data>Traceback (most recent call last):\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/storage/file.py", line 67, in get\n    fh = open(path, "r")\nFileNotFoundError: [Errno 2] No such file or directory: \'POLYGON ((-16 21, -15 21, -15 22, -16 22, -16 21))\'\n\nDuring handling of the above exception, another exception occurred:\n\nTraceback (most recent call last):\n  File "/home/ops/verdi/ops/hysds-0.3.11/hysds/utils.py", line 139, in download_file\n    osaka.main.get(url, cache_dir, params=params)\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 86, in get\n    noclobber=noclobber,\n  File "/home/ops/verdi/ops/osaka-1.0.1/osaka/main.py", line 128, in transfer\n    no